---
* **Universidad:** UOC (Universitat Oberta de Catalunya)
* **Titulación:** Master Ciencia de Datos/TFM
* **Título:** Segmentación de imágenes utilizando Redes Generativas Antagónicas
* **Actividad**: Segmentación partes cuerpo personas con armas cortas utilizando modelos pre entrenados
* **Autor:** Miguel Ponce
* **Correo:** miguelalejoponce@uoc.edu
* **Fecha:** "Enero, 04 de 2023"
* **Salida**: ipynb
---
## Contexto: Detección de personas

* **Contexto:**
Se desea segmentar las personas y armas cortas con los modelos pre entrenados.

* **Problema:** 
Es necesrio realizar la segmentación de las personas armadas con armas cortas, para tener un conjunto de imagenes que luego serán procesadas para ser utilizadas como conjunto de datos para la red GAN.

* **Solucion :** 
Se utilizara los modelos pre entrenados de las etapas anteriores para generar un nuevo conjunto de datos. Para esto se hara uso de los conjuntos de datos tratado con super pixeles ESRGAN y RDN.
* **Entorno:** con GPU.
  

## Origen del la fuente de datos
* **RDN:** https://github.com/miguelalejo/gan-segmentation-person-gun/blob/main/data/raw/super-resolution-rdn.tar.gz

## Modelos pre entrenados
* **DETECCIÓN ARMA:**
https://drive.google.com/file/d/1xAbxae-WI2uiTPUOZvakSNhvbkNBc2sT/view

* **SEGMENTACIÓN ARMA:** https://drive.google.com/file/d/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob/view

* **SEGMENTACIÓN PERSONA:** https://github.com/de-code/python-tf-bodypix

# Descarga conjunto de datos

In [1]:
!wget https://raw.githubusercontent.com/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-rdn.tar.gz  -O super-resolution-rdn.tar.gz 

--2023-01-07 03:18:57--  https://raw.githubusercontent.com/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-rdn.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74354007 (71M) [application/octet-stream]
Saving to: ‘super-resolution-rdn.tar.gz’

super-resolution-rd 100%[===================>]  70.91M   329MB/s    in 0.2s    

2023-01-07 03:19:02 (329 MB/s) - ‘super-resolution-rdn.tar.gz’ saved [74354007/74354007]



In [2]:
!tar -xf super-resolution-rdn.tar.gz

# Deteccion Armas Cortas

## Instalar requisitos

_(Recuerde elegir GPU en tiempo de ejecución si aún no lo ha seleccionado. Tiempo de ejecución --> Cambiar tipo de tiempo de ejecución --> Acelerador de hardware --> GPU)_

In [3]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-5jmw9dm5
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-5jmw9dm5
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9f71f50b13c76e95540a5944bfa00c49db55eab0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 16.9 MB/s eta 0:00:

## Importar Librerias



In [4]:
import os
import glob
import time
import torch
from pathlib import Path
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_hub as hub
import logging
import torch
from collections import OrderedDict
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.utils.events import EventStorage
from detectron2.modeling import build_model
import detectron2.utils.comm as comm
from detectron2.engine import default_argument_parser, default_setup, default_writers, launch
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    inference_on_dataset,
    print_csv_format,
)

## Mostrar caracteristicas del entorno

In [5]:
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
torch:  1.13 ; cuda:  cu116
detectron2: 0.6


## Constantes

In [6]:
# Declaring Constants
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"
TEST_MODE = False
MAX_IMAGES = 100
mainPath = "./"

## Clases


In [7]:
class DetectImage:
  def __init__(self,sourcePath,outputPath,config,modelName,threshold) -> None:
    self.cfg = config
    print(os.path.join(self.cfg.OUTPUT_DIR, modelName))
    self.cfg.MODEL.WEIGHTS = os.path.join(self.cfg.OUTPUT_DIR, modelName)
    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    self.predictor = DefaultPredictor(self.cfg)
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def crop_object(self,imagePath):
    """Crops an object in an image

    Inputs:
      image: PIL image
      box: one box from Detectron2 pred_boxes
    """  
    image = Image.open(imagePath)
    im = cv2.imread(imagePath)
    outputs = self.predictor(im)
    boxes = outputs["instances"].pred_boxes
    # Select 1 box:    
    if list(boxes):
      #print(imagePath)
      #print(boxes,"boxes")
      box = list(boxes)[0].detach().cpu().numpy()         
      x_top_left = box[0]
      y_top_left = box[1]
      x_bottom_right = box[2]
      y_bottom_right = box[3]
      x_center = (x_top_left + x_bottom_right) / 2
      y_center = (y_top_left + y_bottom_right) / 2
      crop_img = image.crop((int(x_top_left), int(y_top_left), int(x_bottom_right), int(y_bottom_right)))
      #plt.figure()
      #plt.axis('off')
      #plt.imshow(crop_img) 
      #plt.show() 
      return True,crop_img,box
    else:
      return False,None,None

  def saveBox(self,box,boxPath):
    with open(boxPath, "w") as f:
      for s in box:
        f.write(str(s) +"\n")
  def readBox(self,boxPath):
    values = []
    with open(boxPath, "r") as f:
      for line in f:
        values.append(float(line.strip()))
    return values
   
  
  def processImages(self):       
    path = self.sourcePath+"/*.jpg"
    print(path)
    images = glob.glob(path)
    index = 0
    start = time.time()
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1             
      hasBox,crop_img,box = self.crop_object(imagePath)
      if hasBox:
        imageName = os.path.basename(imagePath)
        print(imageName)
        name = Path(imagePath).stem        
        boxPath = os.path.join(self.outputPath,"boxes",name+".txt")
        cropImagePath = os.path.join(self.outputPath,"crops",imageName)
        crop_img.save(cropImagePath)
        self.saveBox(box,boxPath)
        print("Time Detect Image : %f" % (time.time() - start))

## Configuracion modelos pre entrenados

In [8]:
from detectron2.engine import DefaultTrainer
instance = "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(instance))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(instance)  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## Descaga modelo detección arma pre-entrenado

In [9]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1" -O ./output/model_final.pth && rm -rf /tmp/cookies.txt

--2023-01-07 03:22:42--  https://docs.google.com/uc?export=download&confirm=t&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1
Resolving docs.google.com (docs.google.com)... 74.125.68.138, 74.125.68.139, 74.125.68.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.68.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5afel0vls14hhjg03esd01q834m9rdun/1673061750000/07156903365772177219/*/1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1?e=download&uuid=af48793b-5d4c-4a47-be37-23d0cda5f93e [following]
--2023-01-07 03:22:43--  https://doc-0g-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5afel0vls14hhjg03esd01q834m9rdun/1673061750000/07156903365772177219/*/1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1?e=download&uuid=af48793b-5d4c-4a47-be37-23d0cda5f93e
Resolving doc-0g-bk-docs.googleusercontent.com (doc-0g-bk-docs.googleusercontent.com)... 142.251.12.132, 2404

## Creacion proyecto deteccion arma RDN

In [10]:
!rm -rf guns-detection-rdn
!mkdir guns-detection-rdn
!mkdir "guns-detection-rdn/crops"
!mkdir "guns-detection-rdn/boxes"
!mkdir "guns-detection-rdn/binmask"
!mkdir "guns-detection-rdn/binredmask"

## Creacion Pipeline red pre entrenada detección arma RDN

In [11]:
sourcePathRDN = os.path.join(mainPath,"super-resolution-rdn/person")
outputPathRDN = os.path.join(mainPath,"guns-detection-rdn")
detectImage = DetectImage(sourcePathRDN,outputPathRDN,cfg,"model_final.pth",0.85)
detectImage.processImages()

./output/model_final.pth
[01/07 03:23:00 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...
./super-resolution-rdn/person/*.jpg


/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


gmd_1_frame75_jpg.rf.2d0952816c9eaa3fd914a0c53ee2edb4.jpg
Time Detect Image : 6.725245
gmd_5_frame6_jpg.rf.e707dcfb14f659fde24df071a0fd448c.jpg
Time Detect Image : 6.958925
gmd_4_frame70_jpg.rf.0c94762ba7f74097359f9ada07ca6e27.jpg
Time Detect Image : 7.187962
gmd_1_frame208_jpg.rf.6d6d76892fb2122d09c8616b7a8a3735.jpg
Time Detect Image : 7.409180
10948387_452160258269315_769342041_n_jpg.rf.37f4964a48434944e9c3d7970b10fe2f.jpg
Time Detect Image : 7.718908
gmd_1_frame22_jpg.rf.64fa6735bb57bf8025fe3aa8d35bb37d.jpg
Time Detect Image : 7.953958
gmd_5_frame20_jpg.rf.6e41657eac837ee7a321835813a0f67e.jpg
Time Detect Image : 8.155238
mgd_custom_frame2770_jpg.rf.2d008f3c8cd6a4f3843c3b9558587ecb.jpg
Time Detect Image : 8.593429
mgd_custom_frame1744_jpg.rf.c17a31c8930b885066fdc84801aa2362.jpg
Time Detect Image : 8.745289
mgd_custom_frame1582_jpg.rf.281c87a6619e93a6b3e41a4e952f0f70.jpg
Time Detect Image : 9.028575
423450861-1607013005_jpeg_jpg.rf.dfdb948014e3c866b610a90f8c016200.jpg
Time Detect Imag

# Segmentacion Armas

## Clases

In [12]:
class SegmentImage:
  def __init__(self,sourcePath,outputPath,config,modelName,threshold) -> None:
    self.cfg = config
    self.cfg.MODEL.WEIGHTS = os.path.join(self.cfg.OUTPUT_DIR, modelName)
    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    self.predictor = DefaultPredictor(self.cfg)
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def crop_shape_object(self,imagePath):
    """Crops an object in an image

    Inputs:
      image: PIL image
      box: one box from Detectron2 pred_boxes
    """  
   
    img= cv2.imread(imagePath)
    outputs= self.predictor(img)
    if outputs['instances']: 
      mask= outputs['instances'].get('pred_masks')    
      mask= mask.to('cpu')    
      num, h, w= mask.shape
      bin_mask= np.zeros((h, w))
      index  = 0
      for m in mask:        
          if index > 0:
            break;
          bin_mask = m.numpy().astype(np.uint8)
          #indices = indices.astype(np.uint8)  #convert to an unsigned byte
          bin_mask*=255
          #print(bin_mask.any(0))
          #print(index)
          index = index +1
          #bin_mask.append(m)
      return True,bin_mask
    return False,None
  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePath+"/*.jpg"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1                
      hasMask,bin_mask = self.crop_shape_object(imagePath)
      if hasMask:
        imageName = os.path.basename(imagePath)
        print(imageName)
        imageNamePng = Path(imageName).stem+'.png'
        im = Image.fromarray(bin_mask)
        imagePngPath = os.path.join(self.outputPath,imageNamePng)                        
        im.save(imagePngPath, "PNG")
        print("Time Segment Image : %f" % (time.time() - start))

## Configuracion modelo segementación pre entrenado

In [13]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("asl_poly_train",)#Train dataset registered in a previous cell
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00055
cfg.SOLVER.MAX_ITER = 1200 
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  
cfg.TEST.EVAL_PERIOD = 0 # Increase this number if you want to monitor validation performance during training

PATIENCE = 500 #Early stopping will occur after N iterations of no imporovement in total_loss

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## Descaga modelo segemntación arma pre-entrenado

In [14]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob" -O ./output/model_final_segmentaion.zip && rm -rf /tmp/cookies.txt

--2023-01-07 03:28:02--  https://docs.google.com/uc?export=download&confirm=t&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob
Resolving docs.google.com (docs.google.com)... 142.251.12.101, 142.251.12.138, 142.251.12.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.12.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/rl4ok42heng272nfh9jks9i5amu31c4g/1673062050000/07156903365772177219/*/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob?e=download&uuid=62d00679-ec11-4ed6-87a9-ac7e754e4955 [following]
--2023-01-07 03:28:04--  https://doc-08-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/rl4ok42heng272nfh9jks9i5amu31c4g/1673062050000/07156903365772177219/*/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob?e=download&uuid=62d00679-ec11-4ed6-87a9-ac7e754e4955
Resolving doc-08-bk-docs.googleusercontent.com (doc-08-bk-docs.googleusercontent.com)... 142.251.12.132, 

In [15]:
!unzip -o ./output/model_final_segmentaion.zip -d ./output/

Archive:  ./output/model_final_segmentaion.zip
  inflating: ./output/model_final_segmentaion.pth  


## Creacion Pipeline red pre entrenada segmentación arma RDN

In [16]:
sourcePathCropsRDN = os.path.join(mainPath,"guns-detection-rdn/crops")
outputPathBinMaskRDN = os.path.join(mainPath,"guns-detection-rdn/binmask")
segmentImage = SegmentImage(sourcePathCropsRDN,outputPathBinMaskRDN,cfg,"model_final_segmentaion.pth",0.60)
segmentImage.processImages()

[01/07 03:28:16 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final_segmentaion.pth ...
gmd_1_frame75_jpg.rf.2d0952816c9eaa3fd914a0c53ee2edb4.jpg
Time Segment Image : 0.208639
gmd_5_frame6_jpg.rf.e707dcfb14f659fde24df071a0fd448c.jpg
Time Segment Image : 0.345414
gmd_4_frame70_jpg.rf.0c94762ba7f74097359f9ada07ca6e27.jpg
Time Segment Image : 0.458285
gmd_1_frame208_jpg.rf.6d6d76892fb2122d09c8616b7a8a3735.jpg
Time Segment Image : 0.563268
10948387_452160258269315_769342041_n_jpg.rf.37f4964a48434944e9c3d7970b10fe2f.jpg
Time Segment Image : 0.704785
gmd_1_frame22_jpg.rf.64fa6735bb57bf8025fe3aa8d35bb37d.jpg
Time Segment Image : 0.815530
mgd_custom_frame1744_jpg.rf.c17a31c8930b885066fdc84801aa2362.jpg
Time Segment Image : 1.188775
mgd_custom_frame1582_jpg.rf.281c87a6619e93a6b3e41a4e952f0f70.jpg
Time Segment Image : 1.292959
423450861-1607013005_jpeg_jpg.rf.dfdb948014e3c866b610a90f8c016200.jpg
Time Segment Image : 1.404442
mgd_varying_frame108_jpg.rf.

# Segemtancion partes del cuerpo

## Instalar requisitos

_(Recuerde elegir GPU en tiempo de ejecución si aún no lo ha seleccionado. Tiempo de ejecución --> Cambiar tipo de tiempo de ejecución --> Acelerador de hardware --> GPU)_

In [17]:
!pip uninstall -y detectron2 
!pip uninstall -y tensorflow
!pip uninstall -y tensorflow_decision_forests
!rm -rf build/ **/*.so

Found existing installation: detectron2 0.6
Uninstalling detectron2-0.6:
  Successfully uninstalled detectron2-0.6
Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Successfully uninstalled tensorflow-2.9.2


In [18]:
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 tf_bodypix opencv-python tfjs_graph_converter matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [19]:
!python -m pip install --upgrade imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Clases

In [20]:
class BodyPix:
  def __init__(self,sourcePath,outputPath) -> None:
    #self.bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_FLOAT_50_STRIDE_16))
    self.bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_RESNET50_FLOAT_STRIDE_16))
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def maskImage(self,imagePath):
    imageName = os.path.basename(imagePath)
    
    # load model (once)

    image = tf.keras.utils.load_img(imagePath)
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    # get prediction result
    #image = tf.keras.preprocessing.image.load_img(fullPath)    
    result = self.bodypix_model.predict_single(image_array)

    # simple mask
    mask = result.get_mask(threshold=0.75)
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-mask",imageName),
        mask)

    # colored mask (separate colour for each body part)
    colored_mask = result.get_colored_part_mask(mask)
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-colored-mask",imageName),
        colored_mask)



    poses = result.get_poses()
    image_with_poses = draw_poses(
        image_array.copy(),  # create a copy to ensure we are not modifing the source image
        poses,
        keypoints_color=(255, 100, 100),
        skeleton_color=(100, 100, 255)
    )
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-poses",imageName),
        image_with_poses
    )
    

  def processImages(self):       
    start = time.time() 
    path = self.sourcePath+"/*.jpg"
    print(path)
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1
      imageName = os.path.basename(imagePath)
      print(imageName)      
      self.maskImage(imagePath) 
      print("Time Taken Body Pix : %f" % (time.time() - start))

## Liberias BodyPix

In [21]:
import tensorflow as tf
from tf_bodypix.api import download_model, load_model, BodyPixModelPaths
from tf_bodypix.draw import draw_poses  # utility function using OpenCV

## Creacion proyecto body pix RDN

In [22]:
!rm -rf body-pix-rdn
!mkdir body-pix-rdn
!mkdir "body-pix-rdn/person"
!mkdir "body-pix-rdn/person/output-mask"
!mkdir "body-pix-rdn/person/output-colored-mask"
!mkdir "body-pix-rdn/person/output-poses"

## Creacion Pipeline red detección partes cuerpo RDN

In [23]:
sourcePathMask = os.path.join(mainPath,"super-resolution-rdn/person")
outputPathBody = os.path.join(mainPath,"body-pix-rdn/person")
bodyPix = BodyPix(sourcePathMask,outputPathBody)
bodyPix.processImages()

./super-resolution-rdn/person/*.jpg
gmd_1_frame75_jpg.rf.2d0952816c9eaa3fd914a0c53ee2edb4.jpg
Time Taken Body Pix : 3.958416
gmd_5_frame6_jpg.rf.e707dcfb14f659fde24df071a0fd448c.jpg
Time Taken Body Pix : 4.737739
gmd_4_frame70_jpg.rf.0c94762ba7f74097359f9ada07ca6e27.jpg
Time Taken Body Pix : 5.499624
gmd_1_frame208_jpg.rf.6d6d76892fb2122d09c8616b7a8a3735.jpg
Time Taken Body Pix : 6.258457
10948387_452160258269315_769342041_n_jpg.rf.37f4964a48434944e9c3d7970b10fe2f.jpg
Time Taken Body Pix : 7.045100
gmd_1_frame22_jpg.rf.64fa6735bb57bf8025fe3aa8d35bb37d.jpg
Time Taken Body Pix : 7.821304
gmd_5_frame20_jpg.rf.6e41657eac837ee7a321835813a0f67e.jpg
Time Taken Body Pix : 7.901182
desi-pistol-image-holding-hands_55_jpeg_jpg.rf.f9c23bbc6f28ef5953b5beb0a5b0658c.jpg
Time Taken Body Pix : 9.709020
mgd_custom_frame2770_jpg.rf.2d008f3c8cd6a4f3843c3b9558587ecb.jpg
Time Taken Body Pix : 10.201734
mgd_custom_frame1744_jpg.rf.c17a31c8930b885066fdc84801aa2362.jpg
Time Taken Body Pix : 10.902228
mgd_custo

# Reemplazo mascara 

## Librerias

In [24]:
import shutil

## Clases

In [25]:
class BitMaskGenerator:
  def __init__(self,sourcePath,outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def createBitMask(self,imagePath):
    img = Image.open(imagePath)
    img = img.convert("RGBA")
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255,0,0, 0))
        else:
            newData.append(item)

    img.putdata(newData)
    return img.convert('RGB')
    

    return False,None
  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePath+"/*.png"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1                      
      rgb_im = self.createBitMask(imagePath)    
      imageName = os.path.basename(imagePath)      
      print(imageName)
      imageNamePng = Path(imageName).stem+'.png'      
      imagePngPath = os.path.join(self.outputPath,imageNamePng) 
      imagePngPath = os.path.join(self.outputPath,imageNamePng)                                
      rgb_im.save(imagePngPath,"PNG")     
      print("Time BitMaskRed Image : %f" % (time.time() - start))

In [26]:
class ImageBitMaskUtil:
  def __init__(self,sourcePath,pathBitMask, pathBitRedMask, pathBox,outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath
    self.pathBitMask = pathBitMask
    self.pathBox = pathBox
    self.pathBitRedMask = pathBitRedMask
    
  def readBox(self,boxPath):
    values = []
    with open(boxPath, "r") as f:
      for line in f:
        values.append(float(line.strip()))
    return values
  def pasteBitMask(self,pathImage,format):
    imageName = os.path.basename(pathImage)
    name  = Path(imageName).stem
    pathImageMask = os.path.join(self.pathBitMask,name+".png")
    pathBitRedMask = os.path.join(self.pathBitRedMask,name+".png")
    if os.path.exists(pathImageMask):
      print(imageName)      
      boxImagePath  = os.path.join(self.pathBox,name+'.txt')
      box = self.readBox(boxImagePath)      
      print("box",box)
      with Image.open(pathImage) as imageBody:
          imageBody.load()
      with Image.open(pathBitRedMask) as imgGunRedMask:
          imgGunRedMask.load()
      with Image.open(pathImageMask) as imgGunMask:
          imgGunMask.load()
    
      imageBody.paste(
          imgGunRedMask.resize((imgGunRedMask.width, imgGunRedMask.height)),
          (int(box[0]) ,int(box[1])),
          imgGunMask.resize((imgGunMask.width, imgGunMask.height)),
      )
      if format == "png":
        imageNamePng = Path(imageName).stem+'.png' 
        pathImageResult = os.path.join(self.outputPath,imageNamePng)
        imageBody.save(pathImageResult,"PNG")
      else:        
        pathImageResult = os.path.join(self.outputPath,imageName)
        imageBody.save(pathImageResult)
  
  def processImages(self,format):   
    start = time.time()    
    path = self.sourcePath+"/*.jpg"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1               
      imageName = os.path.basename(imagePath)      
      self.pasteBitMask(imagePath,format)             
      print("Time PasteBitMaskRed into Image : %f" % (time.time() - start))

In [38]:
class ImageSegmentedUtil:
  def __init__(self,sourcePathSegmented,sourcePath, outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath
    self.sourcePathSegmented = sourcePathSegmented    

  def processImages(self):   
    start = time.time()    
    path = self.sourcePathSegmented+"/*.png"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1        
      imageName = os.path.basename(imagePath)
      imageNameJpg = Path(imageName).stem+'.jpg' 
      print(imageName)
      originalImagePath = os.path.join(self.sourcePath,imageNameJpg) 
      outputImagePath  = os.path.join(self.outputPath,imageNameJpg) 
      shutil.copyfile(originalImagePath, outputImagePath)           
      print("Time PasteBitMaskRed into Image : %f" % (time.time() - start))

## Creacion proyecto merge armas segmentadas y personas

In [39]:
!rm -rf segmented-body-parts-gun-rdn
!mkdir segmented-body-parts-gun-rdn
!mkdir "segmented-body-parts-gun-rdn/original"
!mkdir "segmented-body-parts-gun-rdn/segmentado"
!mkdir "segmented-body-parts-gun-rdn/mascara"

## Creacion Pipeline procesamiento merge partes cuerpo segmentadas y armas

In [40]:
sourcePatBinMask = os.path.join(mainPath,"guns-detection-rdn/binmask")
outputPathBinRedMask = os.path.join(mainPath,"guns-detection-rdn/binredmask")
bitMaskGenerator = BitMaskGenerator(sourcePatBinMask,outputPathBinRedMask)
bitMaskGenerator.processImages()

mgd_varying_frame110_jpg.rf.33fd7ecacac796d031272fb555796ab6.png
Time BitMaskRed Image : 0.040932
mgd_varying_frame222_jpg.rf.0f5f8b607257f02398928c7943acf4242.png
Time BitMaskRed Image : 0.051478
mgd_varying_frame49_jpg.rf.ef40ab199516652820a8612b5d6e7f5b.png
Time BitMaskRed Image : 0.091289
mgd_varying_frame187_jpg.rf.a9df5c41afdd6079f9656b0b91c4cb70.png
Time BitMaskRed Image : 0.111963
425458913-1607521221_jpeg_jpg.rf.7d44e1a91b7b2c4d6863ebdc44c95fac2.png
Time BitMaskRed Image : 0.151439
mgd_custom_frame2660_jpg.rf.e0e6b8134c6682f615b5ab9284f3aa55.png
Time BitMaskRed Image : 0.154228
Untitled-1-33_jpg.rf.4b139f45591b0844387b6bee571a55a1.png
Time BitMaskRed Image : 0.164022
gmd_3_frame42_jpg.rf.afbc5c68395c84c02756f48dedb8b689.png
Time BitMaskRed Image : 0.166416
406057747-1602603154_jpeg_jpg.rf.f3fd29f7ce43653ad1a4628229f8543b.png
Time BitMaskRed Image : 0.177384
mgd_varying_frame45_jpg.rf.714dfc1c002881c5e35828d0b842e650.png
Time BitMaskRed Image : 0.197582
414438953-1604598026_jpe

In [41]:
sourcePathBody = "body-pix-rdn/person/output-colored-mask"
pathBitMask = "guns-detection-rdn/binmask"
pathBitRedMask = "guns-detection-rdn/binredmask"
boxPath = "guns-detection-rdn/boxes"
outputPathSegmented = "segmented-body-parts-gun-rdn/segmentado"
imageBitMaskUtil = ImageBitMaskUtil(sourcePathBody,pathBitMask, pathBitRedMask, boxPath,outputPathSegmented)
imageBitMaskUtil.processImages("jpg")

gmd_1_frame75_jpg.rf.2d0952816c9eaa3fd914a0c53ee2edb4.jpg
box [353.28574, 158.80988, 407.95648, 252.1168]
Time PasteBitMaskRed into Image : 0.022794
gmd_5_frame6_jpg.rf.e707dcfb14f659fde24df071a0fd448c.jpg
box [291.3942, 82.52761, 351.02628, 177.85504]
Time PasteBitMaskRed into Image : 0.034511
gmd_4_frame70_jpg.rf.0c94762ba7f74097359f9ada07ca6e27.jpg
box [1.6388367, 448.73004, 61.21766, 516.96075]
Time PasteBitMaskRed into Image : 0.044631
gmd_1_frame208_jpg.rf.6d6d76892fb2122d09c8616b7a8a3735.jpg
box [10.516304, 298.02704, 73.29729, 354.95578]
Time PasteBitMaskRed into Image : 0.054620
10948387_452160258269315_769342041_n_jpg.rf.37f4964a48434944e9c3d7970b10fe2f.jpg
box [307.82925, 89.80163, 414.46222, 268.56323]
Time PasteBitMaskRed into Image : 0.068159
gmd_1_frame22_jpg.rf.64fa6735bb57bf8025fe3aa8d35bb37d.jpg
box [314.46463, 222.43985, 351.81265, 269.8028]
Time PasteBitMaskRed into Image : 0.079211
Time PasteBitMaskRed into Image : 0.079350
Time PasteBitMaskRed into Image : 0.07941

In [42]:
sourcePathBody = "body-pix-rdn/person/output-mask"
pathBitMask = "guns-detection-rdn/binmask"
pathBitRedMask = "guns-detection-rdn/binmask"
boxPath = "guns-detection-rdn/boxes"
outputPathSegmented = "segmented-body-parts-gun-rdn/mascara"
imageBitMaskUtil = ImageBitMaskUtil(sourcePathBody,pathBitMask, pathBitRedMask, boxPath,outputPathSegmented)
imageBitMaskUtil.processImages("png")

gmd_1_frame75_jpg.rf.2d0952816c9eaa3fd914a0c53ee2edb4.jpg
box [353.28574, 158.80988, 407.95648, 252.1168]
Time PasteBitMaskRed into Image : 0.018555
gmd_5_frame6_jpg.rf.e707dcfb14f659fde24df071a0fd448c.jpg
box [291.3942, 82.52761, 351.02628, 177.85504]
Time PasteBitMaskRed into Image : 0.027499
gmd_4_frame70_jpg.rf.0c94762ba7f74097359f9ada07ca6e27.jpg
box [1.6388367, 448.73004, 61.21766, 516.96075]
Time PasteBitMaskRed into Image : 0.035259
gmd_1_frame208_jpg.rf.6d6d76892fb2122d09c8616b7a8a3735.jpg
box [10.516304, 298.02704, 73.29729, 354.95578]
Time PasteBitMaskRed into Image : 0.042444
10948387_452160258269315_769342041_n_jpg.rf.37f4964a48434944e9c3d7970b10fe2f.jpg
box [307.82925, 89.80163, 414.46222, 268.56323]
Time PasteBitMaskRed into Image : 0.051956
gmd_1_frame22_jpg.rf.64fa6735bb57bf8025fe3aa8d35bb37d.jpg
box [314.46463, 222.43985, 351.81265, 269.8028]
Time PasteBitMaskRed into Image : 0.059703
Time PasteBitMaskRed into Image : 0.059811
Time PasteBitMaskRed into Image : 0.06078

In [43]:
outputOriginalPath = "segmented-body-parts-gun-rdn/original"
imageSegmentedUtil = ImageSegmentedUtil(outputPathSegmented,sourcePathMask, outputOriginalPath)
imageSegmentedUtil.processImages()

mgd_varying_frame110_jpg.rf.33fd7ecacac796d031272fb555796ab6.png
Time PasteBitMaskRed into Image : 0.008592
mgd_varying_frame222_jpg.rf.0f5f8b607257f02398928c7943acf4242.png
Time PasteBitMaskRed into Image : 0.009994
mgd_varying_frame49_jpg.rf.ef40ab199516652820a8612b5d6e7f5b.png
Time PasteBitMaskRed into Image : 0.011624
mgd_varying_frame187_jpg.rf.a9df5c41afdd6079f9656b0b91c4cb70.png
Time PasteBitMaskRed into Image : 0.013134
425458913-1607521221_jpeg_jpg.rf.7d44e1a91b7b2c4d6863ebdc44c95fac2.png
Time PasteBitMaskRed into Image : 0.014625
mgd_custom_frame2660_jpg.rf.e0e6b8134c6682f615b5ab9284f3aa55.png
Time PasteBitMaskRed into Image : 0.015740
Untitled-1-33_jpg.rf.4b139f45591b0844387b6bee571a55a1.png
Time PasteBitMaskRed into Image : 0.016803
gmd_3_frame42_jpg.rf.afbc5c68395c84c02756f48dedb8b689.png
Time PasteBitMaskRed into Image : 0.017863
406057747-1602603154_jpeg_jpg.rf.f3fd29f7ce43653ad1a4628229f8543b.png
Time PasteBitMaskRed into Image : 0.019215
mgd_varying_frame45_jpg.rf.714d

In [44]:
!cp -r guns-detection-rdn segmented-body-parts-gun-rdn

## Compresion Archivos

In [45]:
!rm -rf segmented-body-parts-gun-rdn.tar.gz
!tar -czvf segmented-body-parts-gun-rdn.tar.gz "./segmented-body-parts-gun-rdn"

Se han truncado las últimas 5000 líneas del flujo de salida.
./segmented-body-parts-gun-rdn/guns-detection-rdn/crops/gmd_5_frame29_jpg.rf.0a27e1b452660ce62281a90e28542239.jpg
./segmented-body-parts-gun-rdn/guns-detection-rdn/crops/gmd_3_frame9_jpg.rf.828ae7bc4831795201be2d53247473b4.jpg
./segmented-body-parts-gun-rdn/guns-detection-rdn/crops/mgd_custom_frame1836_jpg.rf.80f574c8bbd49a9c72ae6744946f1d24.jpg
./segmented-body-parts-gun-rdn/guns-detection-rdn/crops/425458919-1607521228_jpeg_jpg.rf.98c3657d743a8a334a248dea84025d43.jpg
./segmented-body-parts-gun-rdn/guns-detection-rdn/crops/442976147-1612242219_jpeg_jpg.rf.15a6d1dc3aaadc253afe00487d0258f6.jpg
./segmented-body-parts-gun-rdn/guns-detection-rdn/crops/mgd_custom_frame2727_jpg.rf.b1849dbe9538309f86bb1c6f4c416359.jpg
./segmented-body-parts-gun-rdn/guns-detection-rdn/crops/gmd_1_frame9_jpg.rf.ddc0e3c051d6c359a34471d25144c095.jpg
./segmented-body-parts-gun-rdn/guns-detection-rdn/crops/girl-posing-guns-23026564_jpg.rf.e6191c7acf43f83f

## Cargar en drive resultados

In [35]:
import os
from google.colab import drive
ROOT_FOLDER='/content/'
ROOT_DRIVE_FOLDER=ROOT_FOLDER+'drive/'
drive.mount(ROOT_DRIVE_FOLDER)

Mounted at /content/drive/


In [46]:
!cp --suffix=_v_$(date +"%y-%m-%d--%T")  ./segmented-body-parts-gun-rdn.tar.gz /content/drive/MyDrive/tmf-person-data/data_set_processed/segmented-body-parts-gun-rdn